In [15]:
# pip install openpyxl ffmpeg


In [16]:
import os
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from IPython.display import HTML
from scipy.interpolate import griddata
from matplotlib.animation import FuncAnimation, PillowWriter

In [17]:
# Load data from the Excel file
data = pd.read_excel("Sensor data.xlsx", sheet_name=None)

In [18]:
data

{'0474D T|RH IP67(0474D-S1)':                 Sensor ID             0474D    Unnamed: 2
 0             Sensor Name   0474D T/RH IP67           NaN
 1            Sensor Group               NaN           NaN
 2                    Time  Temperature (°C)  Humidity (%)
 3     2024-04-16 13:09:53             21.95          50.3
 4     2024-04-16 13:14:52              22.5          34.7
 ...                   ...               ...           ...
 8358  2024-05-15 13:02:14              29.7            43
 8359  2024-05-15 13:07:13             29.55          44.9
 8360  2024-05-15 13:12:13              30.4          42.7
 8361  2024-05-15 13:17:13                31          44.2
 8362  2024-05-15 13:22:12              31.2          42.1
 
 [8363 rows x 3 columns],
 '04750 T|RH IP67(04750-S1)':                  Sensor ID             04750    Unnamed: 2
 0              Sensor Name   04750 T/RH IP67           NaN
 1             Sensor Group               NaN           NaN
 2                     Tim

In [19]:
def extract_data_from_sheet(df):
    sensor_name = df.iloc[0, 1]
    extracted_df = df.iloc[3:].reset_index(drop=True)
    extracted_df.columns = ['Time', 'Temperature', 'Humidity']
    extracted_df['Time'] = pd.to_datetime(extracted_df['Time'], format='%d-%m-%Y %H:%M')
    extracted_df['Sensor Info'] = sensor_name
    extracted_df['Sensor ID'] = extracted_df['Sensor Info'].str[:5]
    return extracted_df

In [20]:
# Extract data from all sheets
data_frames = []
for sheet_name, df in data.items():
    extracted_df = extract_data_from_sheet(df)
    data_frames.append(extracted_df)
combined_data = pd.concat(data_frames, ignore_index=True)

In [21]:
# Handle missing values by filling with the mean value
combined_data['Temperature'] = combined_data['Temperature'].astype('float')
combined_data['Humidity'] = combined_data['Humidity'].astype('float')
combined_data['Temperature'].fillna(combined_data['Temperature'].mean(), inplace=True)
combined_data['Humidity'].fillna(combined_data['Humidity'].mean(), inplace=True)

/var/folders/sd/6m81mt411jxfsqsc4d3ctrg40000gn/T/ipykernel_15189/4201552610.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_data['Temperature'].fillna(combined_data['Temperature'].mean(), inplace=True)
/var/folders/sd/6m81mt411jxfsqsc4d3ctrg40000gn/T/ipykernel_15189/4201552610.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate obje

In [22]:
# Filter and process data
sample_dataset = combined_data[['Time', 'Humidity', 'Sensor ID']]
sensor_ids = sample_dataset['Sensor ID'].unique()

In [23]:
# Define grid size
size_x, size_y = 10, 10

In [24]:
# Create a directory to save the data inspection logs
output_dir = "Sensor_Data"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [25]:
# Plotting heatmaps for each sensor
frames = []
for sensor_id in sensor_ids:
    sensor_data = sample_dataset[sample_dataset['Sensor ID'] == sensor_id].reset_index(drop=True)
    Humidity_grid = np.zeros((size_x, size_y))

    for index, row in sensor_data.iterrows():
        x = index % size_x
        y = index // size_x
        if x < size_x and y < size_y:
            Humidity_grid[x, y] = row['Humidity']

    # Interpolate for smoother look
    xi = np.linspace(0, size_x - 1, size_x * 10)
    yi = np.linspace(0, size_y - 1, size_y * 10)
    xi, yi = np.meshgrid(xi, yi)
    zi = griddata((np.repeat(np.arange(size_x), size_y), np.tile(np.arange(size_y), size_x)),
                  Humidity_grid.flatten(), (xi, yi), method='cubic')

    plt.figure(figsize=(10, 6))
    ax = sns.heatmap(zi, cmap='magma', cbar_kws={'label': 'Humidity (%)'}, fmt=".2f", xticklabels=False, yticklabels=False)
    plt.axis('off')
    plt.tight_layout()

    output_file = os.path.join(output_dir, f"heatmap_{sensor_id}.png")
    plt.savefig(output_file, bbox_inches='tight', pad_inches=0)
    plt.close()

    frames.append(output_file)

print(f"Heat maps saved in the directory: {output_dir}")

Heat maps saved in the directory: Sensor_Data


In [26]:
# Ensure all frames are the same size
first_frame = cv2.imread(frames[0])
height, width, layers = first_frame.shape

def resize_frame(frame, target_size):
    return cv2.resize(frame, (target_size[1], target_size[0]))

resized_frames = []
for frame_path in frames:
    frame = cv2.imread(frame_path)
    resized_frame = resize_frame(frame, (height, width))
    resized_frames.append(resized_frame)

# Create video from heatmap images
video_output = os.path.join(output_dir, "Heatmap_Video.mp4")
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(video_output, fourcc, 5.0, (width, height))

prev_frame = resized_frames[0]

for next_frame in resized_frames[1:]:
    for i in range(1, 10):
        alpha = i / 10.0
        interpolated_frame = cv2.addWeighted(prev_frame, 1 - alpha, next_frame, alpha, 0)
        video.write(interpolated_frame)
    prev_frame = next_frame

video.release()
cv2.destroyAllWindows()

print(f"Video saved as: {video_output}")

Video saved as: Sensor_Data/Heatmap_Video.mp4
